<a href="https://colab.research.google.com/github/Artmagic/GP2_team34/blob/main/%D0%9C%D0%95%D0%A2%D0%A0%D0%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(), options=options)

url = "https://ru.wikipedia.org/wiki/Список_станций_Московского_метрополитена"
driver.get(url)
time.sleep(2)  # имитация ожидания загрузки

# Достаём HTML как бы "через браузер"
html = driver.page_source
driver.quit()

# Парсим как обычный HTML
soup = BeautifulSoup(html, "html.parser")

table = soup.find("table", class_="standard sortable jquery-tablesorter")
data = []

for row in table.find_all("tr")[1:]:
    cols = row.find_all("td")
    if len(cols) < 8:
        continue

    name_tag = cols[2].find("a")
    if not name_tag:
        continue
    name = name_tag.text.strip()

    coord_tag = cols[7].find("a", class_="mw-kartographer-maplink")
    if coord_tag:
        lat = coord_tag.get("data-lat")
        lon = coord_tag.get("data-lon")
    else:
        lat = lon = None

    data.append({
        "station": name,
        "latitude": lat,
        "longitude": lon
    })

df = pd.DataFrame(data)
print("Собрано через Selenium:", len(df))

Собрано через Selenium: 273


In [19]:
df

,station,latitude,longitude
0,Бульвар Рокоссовского,55.8148,37.7342
1,Черкизовская,55.8038,37.7448
2,Преображенская площадь,55.7963,37.7151
3,Сокольники,55.7888,37.6802
4,Красносельская,55.7801,37.6673
...,...,...,...
268,Генерала Тюленева,55.626245,37.486024
269,Тютчевская,55.61891,37.481101
270,Корниловская,55.598782,37.479305
271,Коммунарка,55.574596,37.467965


In [21]:
df = df.drop_duplicates().reset_index(drop=True)
df

,station,latitude,longitude
0,Бульвар Рокоссовского,55.8148,37.7342
1,Черкизовская,55.8038,37.7448
2,Преображенская площадь,55.7963,37.7151
3,Сокольники,55.7888,37.6802
4,Красносельская,55.7801,37.6673
...,...,...,...
261,Генерала Тюленева,55.626245,37.486024
262,Тютчевская,55.61891,37.481101
263,Корниловская,55.598782,37.479305
264,Коммунарка,55.574596,37.467965


In [22]:
df.to_csv("moscow_metro_selenium.csv", index=False, encoding="utf-8-sig")